In [1]:
# The directory for top level folder
dir_ = "/home/sugam/Work/10-19 NLP/12 Projects/Resume Builder/data/"

In [2]:
raw_data_dir = dir_ + 'raw/'
processed_data_dir = dir_+'Processed/'

In [3]:
import pandas as pd
import numpy as np
import re

In [28]:
####################### LOAD DATA
##################################################################
def load_data(url: str) ->pd.DataFrame:
    """
    Loads the dataframe into memory

    Args:
      url (string) - the path to the dataframe

    Returns:
      df (pandas dataframe)
    """
    
    print("⏳ Loading Data")
    
    # Refering our data without any modification and dtype
    if(url[-3:]=="csv"):
        df = pd.read_csv(url)
        
        for col in list(df):  # List out the column names
            if "Unnamed" in col:  # Checks if the column contains Unnamed in it 
                df.drop(col, axis=1,inplace=True) # Removes the Garbage column
    else:
        df = pd.read_json(url, lines=True)


    
    print(f"✅ Done Loading")    
    return df


df_dir= processed_data_dir+"final.csv"

df = load_data(df_dir)

⏳ Loading Data
✅ Done Loading


In [29]:
### Separating the top 1000 for making question answering dataset
df = df[["jobtitle","company","job_role","joblocation","education","experience","skills","industry","payrate"]]
df_1000 = df.iloc[:1000].copy()

In [30]:
df_1000.head()

,jobtitle,company,job_role,joblocation,education,experience,skills,industry,payrate
0,Walkin Data Entry Operator (night Shift),MM Media Pvt Ltd,Fresher Keyskills English Typing Part Time Dat...,Bangalore,Postgraduate,0 - 1 yrs,ITES,Media / Entertainment / Internet,"1,50,000 - 2,25,000 P.A"
1,Work Based Onhome Based Part Time.,find live infotech,Search Engine Marketing/SEM Specialist Keyskil...,Lucknow,M.Tech,0 - 0 yrs,Marketing,Advertising / PR / MR / Event Management,"1,50,000 - 2,50,000 P.A. 20000"
2,Pl/sql Developer - SQL,Softtech Career Infosystem Pvt. Ltd,Software Developer Keyskills SQL Loader PL SQL...,Hyderabad,M.Sc(Science)-Computers,4 - 8 yrs,IT Software - Application Programming,IT-Software / Software Services,Not Disclosed by Recruiter
3,Manager/ad/partner - Indirect Tax - CA,Onboard HRServices LLP,Taxation(Indirect) Manager Keyskills Service T...,Chennai,MS/M.Sc(Science),11 - 15 yrs,Accounts,Banking / Financial Services / Broking,Not Disclosed by Recruiter
4,JAVA Technical Lead (6-8 yrs) -,Spire Technologies and Solutions Pvt. Ltd.,Team Lead/Technical Lead Keyskills Javascript ...,Pune,Diploma-Electronics/Telecommunication,6 - 8 yrs,IT Software - Application Programming,IT-Software / Software Services,Not Disclosed by Recruiter


In [31]:
df_1000 = (
    "JT_" + df_1000["jobtitle"] + \
    ",CO_" + df_1000["company"] +\
    ",JR_" + df_1000["job_role"] + \
    ",JL_" + df_1000["joblocation"] + \
    ",ED_" + df_1000["education"] + \
    ",EX_" + df_1000["experience"] + \
    ",SK_" + df_1000["skills"] + \
    ",IN_" + df_1000["industry"] + \
    ",PR_" + df_1000["payrate"]
)

In [39]:
def remove_garbage(x):
    x = re.sub(r"/",",",str(x)) # Replace \n with nothing
    x = re.sub(r"\xa0","",str(x)) # Replace \n with nothing
    
    return x
    

df_1000 = df_1000.map(remove_garbage)

In [46]:
def find_index(row,df):
    title_index = df[row].find("JT_") + 3
    company_index = df[row].find("CO_") + 3
    job_role_index = df[row].find("JR_") + 3
    job_location_index = df[row].find("JL_") + 3
    education_index = df[row].find("ED_") + 3
    experience_index = df[row].find("EX_") + 3
    skills_index = df[row].find("IN_") + 3
    payrate_index = df[row].find("PR_") + 3

    print(f"title_index {title_index}")
    print(f"company_index {company_index}")
    print(f"job_role_index {job_role_index}")
    print(f"job_location_index {job_location_index}")
    print(f"education_index {education_index}")
    print(f"experience_index {experience_index}")
    print(f"skills_index {skills_index}")
    print(f"payrate_index {payrate_index}")

In [50]:
find_index(10,df_1000)

title_index 3
company_index 35
job_role_index 46
job_location_index 257
education_index 268
experience_index 280
skills_index 302
payrate_index 329


In [51]:
df_1000[10][46:257]

'Sales,Business Development Manager Keyskills Business Development Management Sales Selling Profitability New Business Communication Skills Desired Candidate Profile Please refer to the Job description above ,JL_'

In [53]:
json_objects= []
for index, value in df_1000.items():
    json_objects.append({"context":value})

In [54]:
df2 = pd.DataFrame(json_objects)

In [55]:
# Convert the DataFrame to JSON
json_data = df2.to_json(orient='records')

file_path = processed_data_dir+"first_1000.json"
# Write the JSON data to the file
with open(file_path, 'w') as json_file:
    json_file.write(json_data)